In [1]:
import numpy as np
import os
os.chdir("src")
# !pip install -e .
from nodal_knot import NodalKnot, plot_3D_and_2D_projections

In [2]:
def k_to_zw(kx, ky, kz):
    """ F: 3D Brillouin zone -> C^2 """

    z_real = np.cos(2*kz) + 0.5
    z_imag = np.cos(kx) + np.cos(ky) + np.cos(kz) - 2.0
    z = z_real + 1j*z_imag
    
    w_real = np.sin(kx)
    w_imag = np.sin(ky)
    w = w_real + 1j*w_imag

    return z, w

def zw_to_c_hopf(z, w):
    """ f: C^2 -> C (Hopf Link) """
    return np.power(z, 2) - np.power(w, 2)

def zw_to_c_trefoil(z, w):
    """ f: C^2 -> C (Trefoil Knot) """
    return np.power(z, 2) - np.power(w, 3)

def zw_to_c_figure8(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """
    return np.power(z, 3) - np.power(w, 2)*z

hopf = NodalKnot(k_to_zw, zw_to_c_hopf)
trefoil = NodalKnot(k_to_zw, zw_to_c_trefoil)
figure8 = NodalKnot(k_to_zw, zw_to_c_figure8)

## Skeletons of Non-Thickened Knot

In [3]:
hopf_points = hopf.knot_skeleton_points()
hopf_figure = hopf.plot_3D(hopf_points)
hopf_figure.show()

## Skeletons of Thickened Knot

In [4]:
hopf_points = hopf.knot_skeleton_points(thickness=0.2)
hopf_figure = hopf.plot_3D(hopf_points)
hopf_figure.show()

In [5]:
trefoil_points = trefoil.knot_skeleton_points(thickness=0.2)
trefoil_figure = trefoil.plot_3D(trefoil_points)
trefoil_figure.show()

# Processing with poly2graph

In [6]:
import plotly.graph_objects as go
import networkx as nx


def get_edge_pts(G):
    pts_list = []
    for u, v, data in G.edges(data=True):
        pts = data.get('pts')
        pts_list.append(pts)
    # Combine all edge pts into one array.
    if pts_list:
        all_pts = np.vstack(pts_list)  
    return all_pts
 

def plot_graph(G: nx.Graph) -> go.Figure:
    """
    Create a 3D Plotly visualization of a graph.
    
    This function extracts edge and node data from the graph and creates an interactive 
    3D plot where:
      - Edges are displayed as blue lines.
      - Nodes (with degree ≠ 2) are displayed as red markers.
    
    Parameters:
    -----------
    G : nx.Graph
        The input graph with edge attribute 'pts' containing a NumPy array of shape (N, 3)
        representing the coordinates along each edge, and node attribute 'o' representing the
        node's 3D position.
        
    Returns:
    --------
    fig : go.Figure
        The Plotly figure object for the interactive 3D visualization.
    """
    # Create a list to hold Plotly traces for edges.
    edge_traces = []
    edge_color = 'blue'  # single color for all edges
    for u, v, data in G.edges(data=True):
        pts = data.get('pts')
        if pts is not None:
            if pts.ndim == 2 and pts.shape[1] == 3:
                # Extract x, y, z coordinates for the edge.
                x = pts[:, 0]
                y = pts[:, 1]
                z = pts[:, 2]
                trace = go.Scatter3d(
                    x=x,
                    y=y,
                    z=z,
                    mode='lines',
                    line=dict(color=edge_color, width=2),
                    hoverinfo='none',  # disable hover text
                    showlegend=False   # disable legend entry
                )
                edge_traces.append(trace)
            else:
                print(f"Edge {u}-{v} 'pts' data is not of shape (N, 3).")
        else:
            print(f"Edge {u}-{v} has no 'pts' attribute.")

    # Create a trace for nodes as red points, but only for nodes whose degree is not 2.
    node_positions = {}
    for n in G.nodes():
        if G.degree(n) != 2:
            data = G.nodes[n]
            if 'o' in data:
                node_positions[n] = data['o']

    if node_positions:
        xs = [coord[0] for coord in node_positions.values()]
        ys = [coord[1] for coord in node_positions.values()]
        zs = [coord[2] for coord in node_positions.values()]
        node_trace = go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode='markers',
            marker=dict(size=5, color='red'),
            hoverinfo='none',  # disable hover text
            showlegend=False   # disable legend entry
        )
    else:
        node_trace = None

    # Combine the traces into one Plotly figure.
    data_traces = edge_traces + ([node_trace] if node_trace is not None else [])
    fig = go.Figure(data=data_traces)

    # Update the layout for better viewing and disable the overall legend.
    fig.update_layout(
        title="Interactive 3D Graph Visualization (Nodes with degree ≠ 2)",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        showlegend=False  # disable legend in the layout
    )
    
    return fig  

In [10]:
trefoil = NodalKnot(k_to_zw, zw_to_c_trefoil)
trefoil_points_volume = trefoil.knot_skeleton_points(thickness=0.2,volume_representation=True)
G=trefoil.convert_to_graph(trefoil_points_volume, clean=True)
trefoil_pts_processed=get_edge_pts(G)
trefoil_fig = trefoil.plot_3D(trefoil_pts_processed)
graph_fig =plot_graph(G)
trefoil_fig.show()
graph_fig.show()
trefoil_fig.write_html("./Figures/trefoil_fig.html")
graph_fig.write_html("./Figures/trefoil_fig_withNodes.html")


In [11]:
hopf = NodalKnot(k_to_zw, zw_to_c_hopf)
hopf_points_volume = hopf.knot_skeleton_points(thickness=0.2,volume_representation=True)
G=hopf.convert_to_graph(hopf_points_volume, clean=True)
hopf_pts_processed=get_edge_pts(G)
hopf_fig = hopf.plot_3D(hopf_pts_processed)
graph_fig =plot_graph(G)
hopf_fig.show()
graph_fig.show()
hopf_fig.write_html("./Figures/hopf_fig.html")
graph_fig.write_html("./Figures/hopf_fig_withNodes.html")

In [12]:
figure8 = NodalKnot(k_to_zw, zw_to_c_figure8)
figure8_points_volume = figure8.knot_skeleton_points(thickness=0.09,volume_representation=True)
G=hopf.convert_to_graph(figure8_points_volume, clean=True)
figure8_pts_processed=get_edge_pts(G)
figure8_fig = hopf.plot_3D(figure8_pts_processed)
graph_fig =plot_graph(G)
figure8_fig.show()
graph_fig.show()

figure8_fig.write_html("./Figures/figure8_fig.html")
graph_fig.write_html("./Figures/figure8_fig_withNodes.html")
